In [3]:
import pandas as pd
from prophet import Prophet
import ray
import time

time: 436 µs (started: 2021-09-23 02:41:55 -07:00)


In [4]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 774 µs (started: 2021-09-23 02:42:00 -07:00)


In [5]:
df = pd.read_csv('https://media.githubusercontent.com/media/nishuvr/Dataset/master/test.csv')
items=df['sku'].unique()
stores=df['store'].unique()
df=df.drop(columns=['causal1', 'causal2','price','price.1','temp','return_qty'])
df = df.rename(columns={"date":"ds","demand":"y"})

time: 13.5 s (started: 2021-09-23 02:42:06 -07:00)


In [6]:
groups = df.groupby(['store','sku'])
keys = []
data = []
for key,item in groups:
    keys.append(key)
    data.append(item)

time: 1.47 s (started: 2021-09-23 02:43:01 -07:00)


In [ ]:
import ray
import time
import logging
from prophet import Prophet
import os
ray.init()

logging.getLogger('prophet').setLevel(logging.WARNING)
class suppress_stdout_stderr(object):
    def __init__(self):
        # Open a pair of null files
        self.null_fds = [os.open(os.devnull, os.O_RDWR) for x in range(2)]
        # Save the actual stdout (1) and stderr (2) file descriptors.
        self.save_fds = (os.dup(1), os.dup(2))

    def __enter__(self):
        # Assign the null pointers to stdout and stderr.
        os.dup2(self.null_fds[0], 1)
        os.dup2(self.null_fds[1], 2)

    def __exit__(self, *_):
        # Re-assign the real stdout/stderr back to (1) and (2)
        os.dup2(self.save_fds[0], 1)
        os.dup2(self.save_fds[1], 2)
        # Close the null files
        os.close(self.null_fds[0])
        os.close(self.null_fds[1])
        
@ray.remote
def fit_model(comb):
  with suppress_stdout_stderr():
     model = Prophet()
     model.fit(comb)
  future = model.make_future_dataframe(periods=28)
  forecast = model.predict(future)
  return model,forecast

result=[]
for i in data:
    result.append(fit_model.remote(i))

results = ray.get(result)

In [12]:
def filter_date(result):
    start_date = "2021-04-15"
    end_date = "2021-05-13"
    dataframes=[]
    after_start_date = result["ds"] >= start_date
    before_end_date = result["ds"] <= end_date
    between_two_dates = after_start_date & before_end_date
    filtered_dates = result.loc[between_two_dates]
    return filtered_dates

dataframes=[]
for result in results:  
    filter_data = filter_date(result)
    dataframes.append(filter_data)
    
result = pd.concat(dataframes)
result.to_csv("forecasts.csv")

time: 6.36 s (started: 2021-09-21 05:01:00 -07:00)
